In [ ]:
pip install tensorflow pandas scikit-learn tqdm segmentation-models albumentations

In [1]:
import tensorflow as tf
import os
os.environ['SM_FRAMEWORK'] = 'tf.keras'
import numpy as np
import pandas as pd
import random
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from segmentation_models.losses import dice_loss
from segmentation_models.metrics import iou_score

from GlottisNetV2.Models.GlottisNetV2_e import glottisnetV2_e
from GlottisNetV2.Utils.DataGenerator import DataGenerator
from GlottisNetV2.Utils.data import load_data, metric_mape, mape_ap, mape_pp
from GlottisNetV2.Utils.Callbacks import get_callbacks

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Set paths
coord_train = r"/home/voicelab/Desktop/segmentation_glottis/UNet/points/training/ap.points"  
img_training = r"/home/voicelab/Desktop/segmentation_glottis/BAGLS_ROI/training"  

N_train = 55750 # number of training images

# Create video IDs for training data and save them in Pandas Dataframe
cols = ['z','path']
df_imgs_train = pd.DataFrame(columns= cols)
df_segs_train = pd.DataFrame(columns =cols)

for i in tqdm(range(N_train)):
    row_imgs = {'z' : [i], 'path': [img_training + "/" + str(i) + ".png"]}
    row_segs = {'z': [i], 'path': [img_training + "/" + str(i) + '_seg.png']}
    df_imgs_train = pd.concat([df_imgs_train, pd.DataFrame(row_imgs)])
    df_segs_train = pd.concat([df_segs_train, pd.DataFrame(row_segs)])

print('Created IDs for training images.')

# Save coordinates of anterior and posterior points in Pandas Dataframe
training_data = load_data(coord_train, N_train)
print('Loaded anterior and posterior points to dataframe.')

'''Training'''
# Set random seed for reproducible training
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
rand=np.random.seed(SEED)
random.seed(SEED)
tf.compat.v1.set_random_seed(SEED)

# Set parameters
BATCH_SIZE = 4
FILTERS = 16
LAYERS= 4
LEARNING_RATE = 0.2e-3
EPOCHS = 30
TARGET_HEIGHT = 128
TARGET_WIDTH = 256
SHUFFLE = True
AUGMENT = True
MODEL_PATH = r"/home/voicelab/Desktop/segmentation_glottis/UNet/model/model.h5"
STEPS_PATH = r"/home/voicelab/Desktop/segmentation_glottis/UNet/model/steps"
N_STEPS = 5 # Save every #N_STEPS epoch
RADIUS = 15

with tf.device('/CPU:0'):

    model = glottisnetV2_e(input_size=(TARGET_HEIGHT, TARGET_WIDTH, 1), layers=LAYERS, filters=FILTERS)

# Hard split of training and validation data
    train_imgs, val_imgs, train_segs, val_segs = train_test_split(df_imgs_train,
                                                              df_segs_train,
                                                              test_size = 0.1,
                                                              random_state = SEED)

# Training data --> Augmentation and Shuffle
    training_generator = DataGenerator(train_imgs, train_segs, batch_size = BATCH_SIZE, target_height = TARGET_HEIGHT, \
                                   target_width = TARGET_WIDTH, shuffle = SHUFFLE, df_coordinates = training_data, \
                                   augment = AUGMENT, radius=RADIUS)

# Validation data
    validation_generator = DataGenerator(val_imgs, val_segs, target_height = TARGET_HEIGHT, \
                                     target_width = TARGET_WIDTH, batch_size = BATCH_SIZE, shuffle = False, \
                                     df_coordinates = training_data, augment = False, radius=RADIUS)


# Compile model with dice_loss for segmentation, mse for prediction maps and use Adam as optimizer
# First exit: predictions of anterior and posterior points (2 channels)
# Second exit: Segmentations
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = LEARNING_RATE), \
              metrics = {'seg': ['acc', iou_score],
                         'ap_pred': ['acc', metric_mape, mape_ap, mape_pp]},
              loss = {'ap_pred': 'mse', 'seg': dice_loss}, run_eagerly=True)




# Train model on dataset and save it
    model.fit(training_generator, validation_data= validation_generator, epochs = EPOCHS,
                    callbacks = get_callbacks(MODEL_PATH, model, N_STEPS, STEPS_PATH))



2024-09-14 13:39:31.572007: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-14 13:39:31.592437: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-14 13:39:31.592458: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-14 13:39:31.593021: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-14 13:39:31.596648: I tensorflow/core/platform/cpu_feature_guar

Segmentation Models: using `tf.keras` framework.


/home/voicelab/miniconda3/envs/tmpenv/lib/python3.11/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/voicelab/miniconda3/envs/tmpenv/lib/python3.11/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.15 (you have 1.4.14). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


  0%|          | 0/55750 [00:00<?, ?it/s]

Created IDs for training images.


...

2024-09-14 13:40:37.074585: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-14 13:40:37.093026: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-14 13:40:37.093145: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Epoch 1/30


2024-09-14 13:40:38.538421: I external/local_xla/xla/service/service.cc:168] XLA service 0x100905d0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-09-14 13:40:38.538437: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2024-09-14 13:40:38.545135: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1726332038.576857 2424388 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2024-09-14 13:40:38.577542: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.


MAPE [-1. -1. -1. -1.]
AP [-1. -1. -1. -1.]
PP [-1. -1. -1. -1.]
    1/12543 [..............................] - ETA: 12:03:37 - loss: 1.1525 - ap_pred_loss: 0.2803 - seg_loss: 0.8722 - ap_pred_acc: 0.9077 - ap_pred_metric_mape: 1.0000 - ap_pred_mape_ap: 1.0000 - ap_pred_mape_pp: 1.0000 - seg_acc: 0.3704 - seg_iou_score: 0.0683MAPE [-1. -1. -1. -1.]
AP [-1. -1. -1. -1.]
PP [-1. -1. -1. -1.]
    2/12543 [..............................] - ETA: 1:30:48 - loss: 1.1813 - ap_pred_loss: 0.2781 - seg_loss: 0.9032 - ap_pred_acc: 0.9041 - ap_pred_metric_mape: 1.0000 - ap_pred_mape_ap: 1.0000 - ap_pred_mape_pp: 1.0000 - seg_acc: 0.3515 - seg_iou_score: 0.0511 MAPE [-1. -1. -1. -1.]
AP [-1. -1. -1. -1.]
PP [-1. -1. -1. -1.]
    3/12543 [..............................] - ETA: 1:32:29 - loss: 1.1919 - ap_pred_loss: 0.2774 - seg_loss: 0.9145 - ap_pred_acc: 0.9038 - ap_pred_metric_mape: 1.0000 - ap_pred_mape_ap: 1.0000 - ap_pred_mape_pp: 1.0000 - seg_acc: 0.3496 - seg_iou_score: 0.0449MAPE [-1. -1. -1.

KeyboardInterrupt: 